In [6]:
import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
from tabulate import tabulate

#### Criar o DataFrame

In [25]:
df = pd.read_parquet("raw_data")

In [26]:
df.shape

(326910, 113)

#### Identificar campos com valores constantes em todos os registros

In [67]:
same_values = [
    column for column in df.columns 
    if len(df[column].unique()) <= 1
    ]

In [ ]:
print(tabulate([[column, list(df[column].unique())] for column in same_values],
               headers=["Campo", "Valor Constante"], tablefmt="pretty"))

+------------+-------------------+
|   Campo    |  Valor Contante   |
+------------+-------------------+
| UTI_MES_IN |      [' 0']       |
| UTI_MES_AN |      [' 0']       |
| UTI_MES_AL |      [' 0']       |
| UTI_INT_IN |      [' 0']       |
| UTI_INT_AN |      [' 0']       |
| UTI_INT_AL |      [' 0']       |
|  VAL_SADT  | ['         0.00'] |
|   VAL_RN   | ['         0.00'] |
| VAL_ACOMP  | ['         0.00'] |
|  VAL_ORTP  | ['         0.00'] |
| VAL_SANGUE | ['         0.00'] |
| VAL_SADTSR |  ['       0.00']  |
| VAL_TRANSP | ['         0.00'] |
| VAL_OBSANG |  ['       0.00']  |
| VAL_PED1AC |  ['       0.00']  |
| DIAG_SECUN |     ['0000']      |
|  NATUREZA  |      ['00']       |
|  RUBRICA   |     ['    0']     |
|  NUM_PROC  |       ['']        |
| TOT_PT_SP  |    ['     0']     |
|  CPF_AUT   |       ['']        |
|  SEQ_AIH5  |      ['000']      |
|    CBOR    |    ['000000']     |
|   CNAER    |      ['000']      |
|  VINCPREV  |       ['0']       |
| GESTOR_DT  |      

#### Identificar campos com prevalência de valores supeior a 95%


In [100]:
prevalent_values=[
    column for column in df.columns
    if column != 'COD_IDADE' and column !="NACIONAL"
    and column not in same_values
    and ((df[column].value_counts(normalize=True).iloc[0]) * 100) > 95
    ]

In [101]:
print(tabulate(
    [[column,
    round((df[column].value_counts(normalize=True).iloc[0]) * 100, 4),
    df[column].value_counts().idxmax()]
    for column in prevalent_values],
    headers=["Campo", "% Prevalente", "Valor Prevalente"],
    tablefmt="pretty"
    ))

+------------+--------------+------------------+
|   Campo    | % Prevalente | Valor Prevalente |
+------------+--------------+------------------+
| UTI_MES_TO |   99.9043    |        0         |
| MARCA_UTI  |   99.9043    |        00        |
| UTI_INT_TO |   99.9994    |        0         |
| DIAR_ACOM  |   96.1405    |        0         |
|  VAL_UTI   |   99.9043    |       0.00       |
|  IND_VDRL  |   99.9419    |        0         |
|   MORTE    |   99.8498    |        0         |
|  HOMONIMO  |   99.9893    |        0         |
| NUM_FILHOS |   99.9997    |        0         |
|   INSTRU   |   99.9997    |        0         |
| CID_NOTIF  |   99.9997    |                  |
| CONTRACEP1 |   99.9997    |        00        |
| CONTRACEP2 |   99.9997    |        00        |
| GESTRISCO  |   99.9997    |        1         |
|  INSC_PN   |   99.9985    |   000000000000   |
| GESTOR_COD |   99.2392    |      00000       |
|  COMPLEX   |   99.9979    |        02        |
|   FINANC   |   99.

In [137]:
df[df.columns.difference(same_values + prevalent_values)]

,ANO_CMPT,CAR_INT,CEP,CGC_HOSP,CNES,CNPJ_MANT,COBRANCA,COD_IDADE,DIAGSEC1,DIAG_PRINC,DIAS_PERM,DT_INTER,DT_SAIDA,ESPEC,GESTAO,GESTOR_CPF,GESTOR_TP,IDADE,IDENT,MES_CMPT,MUNIC_MOV,MUNIC_RES,NACIONAL,NASC,NAT_JUR,N_AIH,PROC_REA,PROC_SOLIC,QT_DIARIAS,RACA_COR,REGCT,REMESSA,SEQUENCIA,SEXO,TPDISEC1,UF_ZI,US_TOT,VAL_SH,VAL_SP,VAL_TOT
4,2019,02,08570390,46374500013504,2078562,46374500000194,12,4,,F192,2,20190116,20190118,05,2,000000000000000,0,21,1,01,352310,352310,010,19971208,1023,3519101604414,0303170140,0303170140,2,03,7102,HE35000001N201901.DTS,62344,1,0,350000,30.56,99.56,14.44,114.00
5,2019,02,08576500,46374500013504,2078562,46374500000194,12,4,F059,F200,14,20190117,20190131,05,2,000000000000000,0,59,1,01,352310,352310,010,19590826,1023,3519101604733,0303170140,0303170140,14,03,7102,HE35000001N201901.DTS,62345,1,1,350000,240.06,794.36,101.08,895.44
6,2019,02,08576510,46374500013504,2078562,46374500000194,12,4,F209,F192,7,20190117,20190124,05,2,000000000000000,0,27,1,01,352310,352310,010,19910417,1023,3519101604766,0303170140,0303170140,7,02,7102,HE35000001N201901.DTS,62346,1,1,350000,106.97,348.46,50.54,399.00
7,2019,02,08595850,46374500013504,2078562,46374500000194,12,4,F200,F312,9,20190117,20190126,05,2,000000000000000,0,56,1,01,352310,352310,010,19620707,1023,3519101604887,0303170140,0303170140,9,03,7102,HE35000001N201901.DTS,62347,3,1,350000,163.65,545.46,64.98,610.44
8,2019,02,08595500,46374500013504,2078562,46374500000194,12,4,F192,F312,13,20190117,20190130,05,2,000000000000000,0,36,1,01,352310,352310,010,19820905,1023,3519101604953,0303170140,0303170140,13,02,7102,HE35000001N201901.DTS,62348,1,1,350000,198.65,647.14,93.86,741.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224309,2024,02,09720375,,2069776,46523239000147,31,4,,F29,4,20241217,20241221,05,1,000000000000000,0,47,1,12,354870,354870,010,19770428,1244,3524132785239,0303170140,0303170140,5,03,0000,HM35487001N202412.DTS,1686,1,0,354870,55.84,288.90,36.10,325.00
224414,2024,02,09720375,,2069776,46523239000147,31,4,,F208,3,20241211,20241214,03,1,000000000000000,0,31,1,12,354870,354870,010,19930914,1244,3524132782731,0303170140,0303170140,4,03,0000,HM35487001N202412.DTS,1220,1,0,354870,39.17,199.12,28.88,228.00
224769,2024,02,09761243,,2069776,46523239000147,31,4,,F195,11,20241206,20241217,03,1,000000000000000,0,25,1,12,354870,354870,010,19990806,1244,3524132782863,0303170140,0303170140,12,03,0000,HM35487001N202412.DTS,1230,1,0,354870,117.52,597.36,86.64,684.00
224773,2024,02,09862060,,2069776,46523239000147,31,4,,F29,6,20241212,20241218,03,1,000000000000000,0,59,1,12,354870,354870,010,19650927,1244,3524132784469,0303170140,0303170140,7,01,0000,HM35487001N202412.DTS,1339,3,0,354870,82.50,429.66,50.54,480.20


#### Analisar Usabilidade dos campos

#### Investigar possível campo agrupador.

In [118]:
df.groupby('N_AIH').size()[df.groupby('N_AIH').size() > 70].sort_values(ascending=False)

N_AIH
3514108696670    73
3508102981735    72
3514104691812    72
3517124637488    72
3517101777684    72
3516100316806    72
3515112099378    72
3515108643574    72
3514119891250    72
3514119890919    72
3514119890655    72
3514119890622    72
3514119890457    72
3514119890435    72
3508102982120    72
3514108687640    72
3513113087583    72
3512120461773    72
3508103318896    72
3508103320348    72
3508103322383    72
3508103328708    72
3508103328719    72
3508103679476    72
3518134814964    72
3508118574939    72
3510107087652    72
3511114005872    72
3511118446902    72
3512100397730    72
3512120461762    72
3514108704030    71
3518134814667    71
3514119890996    71
dtype: int64

In [128]:
df[df['N_AIH'] == '3514108696670'][['N_AIH', 'MES_CMPT', 'ANO_CMPT', 'DT_INTER', 'DT_SAIDA', 'IDADE', 'SEXO', 'NASC']]

,N_AIH,MES_CMPT,ANO_CMPT,DT_INTER,DT_SAIDA,IDADE,SEXO,NASC
52852,3514108696670,01,2019,20140801,20181130,54,3,19640613
52853,3514108696670,01,2019,20140801,20190131,54,3,19640613
187175,3514108696670,02,2019,20140801,20190228,54,3,19640613
189835,3514108696670,03,2019,20140801,20190331,54,3,19640613
134397,3514108696670,04,2019,20140801,20190430,54,3,19640613
...,...,...,...,...,...,...,...,...
147212,3514108696670,08,2024,20140801,20240831,60,3,19640613
127775,3514108696670,09,2024,20140801,20240930,60,3,19640613
153830,3514108696670,10,2024,20140801,20241031,60,3,19640613
210076,3514108696670,11,2024,20140801,20241130,60,3,19640613


In [129]:
df[df['N_AIH'] == '3508102981735'][['N_AIH', 'MES_CMPT', 'ANO_CMPT', 'DT_INTER', 'DT_SAIDA', 'IDADE', 'SEXO', 'NASC']]

,N_AIH,MES_CMPT,ANO_CMPT,DT_INTER,DT_SAIDA,IDADE,SEXO,NASC
36702,3508102981735,01,2019,20080101,20190131,75,1,19430110
190840,3508102981735,02,2019,20080101,20190228,76,1,19430110
201286,3508102981735,03,2019,20080101,20190331,76,1,19430110
56186,3508102981735,04,2019,20080101,20190430,76,1,19430110
73899,3508102981735,05,2019,20080101,20190531,76,1,19430110
...,...,...,...,...,...,...,...,...
148660,3508102981735,08,2024,20080101,20240831,81,1,19430110
138909,3508102981735,09,2024,20080101,20240930,81,1,19430110
31341,3508102981735,10,2024,20080101,20241031,81,1,19430110
55956,3508102981735,11,2024,20080101,20241130,81,1,19430110


In [130]:
df[df['N_AIH'] == '3517124637488'][['N_AIH', 'MES_CMPT', 'ANO_CMPT', 'DT_INTER', 'DT_SAIDA', 'IDADE', 'SEXO', 'NASC']]

,N_AIH,MES_CMPT,ANO_CMPT,DT_INTER,DT_SAIDA,IDADE,SEXO,NASC
135893,3517124637488,01,2019,20171120,20190131,80,1,19381209
192273,3517124637488,02,2019,20171120,20190228,80,1,19381209
189566,3517124637488,03,2019,20171120,20190331,80,1,19381209
119248,3517124637488,04,2019,20171120,20190430,80,1,19381209
217368,3517124637488,05,2019,20171120,20190531,80,1,19381209
...,...,...,...,...,...,...,...,...
190502,3517124637488,08,2024,20171120,20240831,85,1,19381209
109012,3517124637488,09,2024,20171120,20240930,85,1,19381209
122483,3517124637488,10,2024,20171120,20241031,85,1,19381209
176553,3517124637488,11,2024,20171120,20241130,85,1,19381209


#### Validar N_AIH como campo agrupador

In [142]:
# Analisar se existe mais de um valor para o campo DT_INTER para cada N_AIH
(df.groupby('N_AIH')['DT_INTER'].nunique() == 1).all()

True

In [143]:
# Analisar se existe mais de um valor para o campo DIAG_PRINC para cada N_AIH
(df.groupby('N_AIH')['DIAG_PRINC'].nunique() == 1).all()

True

In [131]:
# Analisar se existe mais de um valor para o campo Nasc para cada N_AIH
(df.groupby('N_AIH')['NASC'].nunique() == 1).all()

False

In [ ]:
# Analisar se existe mais de um valor para o campo SEXO para cada N_AIH
(df.groupby('N_AIH')['SEXO'].nunique() == 1).all()

False

In [144]:
# Analisar se existe mais de um valor para o campo RACA_COR para cada N_AIH
(df.groupby('N_AIH')['RACA_COR'].nunique() == 1).all()

False

In [133]:
# Analisar se existe mais de um valor para o campo MUNIC_RES para cada N_AIH
(df.groupby('N_AIH')['MUNIC_RES'].nunique() == 1).all()

False

In [140]:
# Analisar se existe mais de um valor para o campo ESPEC para cada N_AIH
(df.groupby('N_AIH')['ESPEC'].nunique() == 1).all()

True

In [141]:
# Analisar se existe mais de um valor para o campo IDENT para cada N_AIH
(df.groupby('N_AIH')['IDENT'].nunique() == 1).all()

False

In [146]:
[(col, (df.groupby('N_AIH')[col].nunique() == 1).all())  for col in df.columns if col not in (same_values + prevalent_values)]

[('UF_ZI', True),
 ('ANO_CMPT', False),
 ('MES_CMPT', False),
 ('ESPEC', True),
 ('CGC_HOSP', True),
 ('N_AIH', True),
 ('IDENT', False),
 ('CEP', False),
 ('MUNIC_RES', False),
 ('NASC', False),
 ('SEXO', False),
 ('QT_DIARIAS', False),
 ('PROC_SOLIC', True),
 ('PROC_REA', True),
 ('VAL_SH', False),
 ('VAL_SP', False),
 ('VAL_TOT', False),
 ('US_TOT', False),
 ('DT_INTER', True),
 ('DT_SAIDA', False),
 ('DIAG_PRINC', True),
 ('COBRANCA', False),
 ('NAT_JUR', True),
 ('GESTAO', True),
 ('MUNIC_MOV', True),
 ('COD_IDADE', False),
 ('IDADE', False),
 ('DIAS_PERM', False),
 ('NACIONAL', False),
 ('CAR_INT', False),
 ('GESTOR_TP', False),
 ('GESTOR_CPF', False),
 ('CNES', True),
 ('CNPJ_MANT', True),
 ('REGCT', False),
 ('RACA_COR', False),
 ('SEQUENCIA', False),
 ('REMESSA', False),
 ('DIAGSEC1', False),
 ('TPDISEC1', False)]

In [147]:
(df.groupby('N_AIH').filter(lambda x: x['MUNIC_RES'].nunique() != 1)['N_AIH'].value_counts()).count()

98